In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import itertools
plt.style.use('ggplot')

In [2]:
df = pd.read_csv("alldata3.csv")
df['reviews'] = df['reviews'].fillna(0) #replace null values with 0.
df.head()
df = df.dropna(axis=0, how = 'any') #remove rows with all null values
sum(df.isnull().any(axis=1)) # check if the removal is successful

0

In [3]:
newlocation = [str(row).rstrip() for row in df['location']]
newlocation = [re.sub('[\s*]+[\d]+[\d]+[\d]+[\d]+[\d]','',row) for row in newlocation]
df['newlocation'] = pd.Series(newlocation,index = df.index)
df['newlocation'].unique() # Check


array(['Atlanta, GA', 'Austin, TX', 'Round Rock, TX', 'Cedar Park, TX',
       'Boulder, CO', 'Boston, MA', 'Chicago, IL', 'Washington, DC',
       'Los Angeles, CA', 'Cambridge, MA', 'Mountain View, CA',
       'Manhattan, NY', 'New York, NY', 'Jersey City, NJ', 'Newark, NJ',
       'Oakland, NJ', 'Bronx, NY', 'Hoboken, NJ', 'Brooklyn, NY',
       'Rutherford, NJ', 'Union, NJ', 'Rahway, NJ', 'Murray Hill, NJ',
       'South Plainfield, NJ', 'Queens, NY', 'Little Falls, NJ',
       'Florham Park, NJ', 'Troy Hills, NJ', 'Bloomfield, NJ',
       'Lynbrook, NY', 'Allendale, NJ', 'Parsippany, NJ',
       'West Orange, NJ', 'Kenilworth, NJ', 'Ridgefield, NJ',
       'Harrison, NY', 'Montclair, NJ', 'Fort Lee, NJ', 'Carlstadt, NJ',
       'Orangeburg, NY', 'Fairfield, NJ', 'Palisades, NY',
       'Hackensack, NJ', 'Roseland, NJ', 'East Hanover, NJ',
       'Long Beach, NY', 'Whippany, NJ', 'Manhasset, NY', 'Yonkers, NY',
       'New Hyde Park, NY', 'Secaucus, NJ', 'Port Washington, NY',
    

In [4]:
city = [re.split(',',row)[0] for row in df['newlocation']]
state = [re.split(',',row)[1] for row in df['newlocation']]
state = map(lambda x: x.strip(),state)
df['city'] = pd.Series(city,index = df.index)
df['state'] = pd.Series(state,index = df.index)

In [5]:
df['description'] = df['description'].apply(lambda x: re.sub('[^\w\s]','', x))

In [7]:
df.to_excel('cleandata.xlsx')

In [8]:
# Put all string together
all_job_description = ""
for job in df['description']:
    all_job_description += " " + job

In [9]:
# Remove stop words
from nltk.corpus import stopwords
stop_words = set((stopwords.words('english')))

In [10]:
from nltk.tokenize import word_tokenize
all_job_description_l = all_job_description.lower()
tokens = word_tokenize(str(all_job_description_l))
len(tokens)

3594374

In [11]:
filtered_description = []
for word in tokens:
    if word not in list(stop_words):
        filtered_description += [word]

In [26]:
filtered_description_string = ""
for token in filtered_description:
    filtered_description_string += ' ' + token

In [14]:
from collections import Counter
Counter(filtered_description).most_common()

[('data', 39434),
 ('experience', 31461),
 ('work', 18951),
 ('team', 15981),
 ('research', 14200),
 ('skills', 12593),
 ('development', 12423),
 ('business', 11787),
 ('ability', 10107),
 ('learning', 9630),
 ('science', 9126),
 ('years', 9053),
 ('new', 8943),
 ('analysis', 8917),
 ('including', 8344),
 ('management', 7956),
 ('working', 7891),
 ('design', 7882),
 ('technical', 7828),
 ('support', 7500),
 ('required', 7493),
 ('software', 7453),
 ('machine', 7378),
 ('strong', 7362),
 ('product', 7249),
 ('engineering', 6854),
 ('knowledge', 6797),
 ('systems', 6703),
 ('related', 6439),
 ('degree', 6417),
 ('environment', 6346),
 ('teams', 6319),
 ('develop', 6180),
 ('solutions', 6115),
 ('analytics', 5851),
 ('information', 5681),
 ('qualifications', 5663),
 ('requirements', 5593),
 ('opportunity', 5538),
 ('position', 5529),
 ('company', 5511),
 ('us', 5410),
 ('technology', 5347),
 ('products', 5285),
 ('projects', 5140),
 ('scientists', 5046),
 ('status', 4924),
 ('using', 4918

In [113]:
tool_list = ['python','sql','java','scala','excel',
               'aws','hadoop','spark','linux','tableau','sas',
               'linux','sas','hive','oracle','perl',
               'mysql','matlab','html','tensorflow','spss','mongodb']
skills_list = ['machine learning','data analysis','data visualization',
            'modeling','statistical analysis','research','deep learning',
            'optimization','decision tree','logistic','random forest',
            'ab testing','web scraping','neural network','interpersonal skills',
            'communication skills','artificial intelligence']
degree_vector = []
for string in df['description']:
    string = str(string).lower()
    if 'degree' in string:
        index = string.index('degree')
        if index < 100:
            degree_vector.append(string[0:index+100])
        else:
            degree_vector.append(string[index-100:index+100])
    else:
        degree_vector.append('No degree requirement')

degree_list = ['master',"masters",'phd','bachelor',"bachelors",
             'high school','No degree requirement']

major_list = ['computer science','data science','statistics','quantitative',
            'mathematics','economics','liberal arts','science','buisness',
            'engineer','engineering','psychology','biology','medicine',
            'marketing','business analysis','chemical engineering','chemistry',
            'public health','public relationship']

In [16]:
def countkeywords(keyword,text):
    num = [str(row).count(keyword) for row in text]
    return keyword,sum(num)

In [114]:
keywords = []
tool_word_count = []
for keyword in tool_list:
    keywords.append(countkeywords(keyword,filtered_description)[0])
    tool_word_count.append(countkeywords(keyword,filtered_description)[1])

In [115]:
for keyword in ['r','bi','c','redshift','scikit']:
    keywords.append(keyword)
    tool_word_count.append(filtered_description.count(keyword))

In [118]:
tool = pd.DataFrame()
tool['keyword'] = keywords
tool['word_count'] = tool_word_count

In [119]:
#import sys 
#reload(sys)
#sys.setdefaultencoding('utf8')
tool.to_excel('tool.xlsx',sheet_name = 'sheet1')

In [70]:
skill_word_count = []
for skill_keyword in skills_list:
    skill_word_count.append(filtered_description_string.count(skill_keyword))
skill = pd.DataFrame()
skill['keyword'] = skills_list
skill['word_count'] = skill_word_count

In [71]:
skill_word_count2 = []
for skill_keyword in ['ai','nlp','ml','bi']:
    skill_word_count2.append(filtered_description.count(skill_keyword))
additional_skill_keyword = ['ai','nlp','ml','bi']
additional = [[word,filtered_description.count(word)] for word in additional_skill_keyword]

In [256]:
filtered_description.count('spark')

1084

In [72]:
skill2 = pd.DataFrame(additional,columns = ['keyword','word_count'])

In [73]:
skill = skill.append(skill2)

In [74]:
skill.to_excel('skill.xlsx',sheet_name = 'sheet1')

In [95]:
degree_word_count = []
for degree in degree_list:
    degree_word_count.append(countkeywords(degree,degree_vector)[1])

In [96]:
# Tokenize degree_vector to count abbrevations of degrees.
from nltk.tokenize import word_tokenize
tokenized_degree_vector = [word_tokenize(degree) for degree in degree_vector]
for degree in ['ba','bs','ma','ms','undergraduate','graduate']:
    degree_list.append(degree)
    degree_word_count.append(countkeywords(degree,tokenized_degree_vector)[1])
degree = pd.DataFrame()

In [99]:
degree['keyword'] = degree_list
degree['word_count'] = degree_word_count
degree.to_excel('degree.xlsx',sheet_name = 'sheet1')

In [103]:
major_word_count = []
for major in major_list:
    major_word_count.append(countkeywords(major,degree_vector)[1])

In [102]:
major = pd.DataFrame()
major['keyword'] = major_list
major['word_count'] = major_word_count
major.to_excel('major.xlsx',sheet_name = 'sheet1')

In [ ]:
# Compare by title

In [128]:
lower_position = [position.lower() for position in df['position']]
temp_counter = Counter(lower_position).most_common()

In [187]:
category = []
for position in lower_position:
    if "data scientist" in position or 'data science' in position or "machine learning" in position:
        category.append('data scientist')
    elif "analyst" in position or "analytics" in position:
        category.append('analyst')
    elif "engineer" in position or "artificial intelligence" in position or "ai" in position:
        category.append('engineer')
    else:
        category.append('other')

In [188]:
df['title_category'] = category

In [189]:
Counter(category).most_common()

[('other', 2883),
 ('data scientist', 1919),
 ('engineer', 1113),
 ('analyst', 961)]

In [ ]:
# Compare by company size

In [138]:
size_category = []
for review in df['reviews']:
    if review <= 50:
        size_category.append('small')
    elif review <= 700:
        size_category.append('medium')
    else:
        size_category.append('large')

In [173]:
df['size'] = size_category

In [139]:
Counter(size_category).most_common()

[('small', 3267), ('medium', 1919), ('large', 1690)]

In [140]:
# Generate a new dataframe to put all keywords and frequencies in it. 

In [155]:
all_keywords = tool_list + skills_list + degree_list + major_list

In [159]:
len(all_keywords)

66

In [170]:
def word_frequency(keyword):
    temp = []
    for single in df['description']:
        single = single.lower()
        temp.append(single.count(keyword))
    return temp

In [175]:
def add_column(word):
    df[word] = word_frequency(word)

In [180]:
for word in all_keywords:
    add_column(word)

In [190]:
df.to_csv("final.csv")

2

In [202]:
# Tokenize all string to count the abbreviation words.

In [201]:
tokenized_description= [word_tokenize(str(single).lower()) for single in df['description']] 
len(tokenized_description)

6876

In [204]:
addtional_keywords = ['ai','ml','bi','r','ba','bs','ma','ms']

In [220]:
def addtional_word_frequency(keyword):
    temp = []
    for single in tokenized_description:
        temp.append(single.count(keyword))
    return temp

In [228]:
def add_additional_column(word):
    df[word] = addtional_word_frequency(word)

In [230]:
for word in addtional_keywords:
    add_additional_column(word)

In [232]:
copy = df

In [239]:
list(copy)
copy['artificial intelligence'] += copy['ai']
copy['sql'] += copy['mysql']
copy['machine learning'] += copy['ml']
copy['master'] += (copy['master'] + copy['ma'] + copy['ms'])

In [240]:
copy['bachelor'] += (copy['bachelors'] + copy['ba'] + copy['bs'])
copy['engineer'] += copy['engineering']

In [246]:
copy = copy.drop(columns = ['ai','mysql','ml','master','ma','ms','bachelors','ba','bs','engineering','ba/bs'])

In [247]:
copy.to_csv('finalclean.csv')

In [250]:
copy2 = copy.drop(columns = ['bi','linux','tableau','hive','oracle','sas',
                            'hive','oracle','perl','matlab','html','tensorflow','spss','mongodb',
                            'research','ab testing','web scraping',
                            'neural network','interpersonal skills','communication skills',
                            'data science','liberal arts','buisness','psychology',
                            'biology','medicine','marketing','business analysis',
                            'chemical engineering','chemistry','public health','public relationship'])

In [269]:
list(copy2)[25:60]

['phd',
 'bachelor',
 'high school',
 'No degree requirement',
 'computer science',
 'statistics',
 'quantitative',
 'mathematics',
 'economics',
 'science',
 'engineer',
 'r']

In [262]:
copy2 = copy2.drop(columns = ['aws','hadoop',])

In [263]:
copy2['machine learning'] += (copy2['deep learning'] + copy2['decision tree'] 
                              + copy2['logistic'] + copy2['random forest'])

In [264]:
copy2['data analysis'] += copy2['statistical analysis']

In [266]:
copy2 = copy2.drop(columns = ['deep learning','decision tree','logistic','random forest'])

In [270]:
copy2['statistics'] += (copy2['quantitative'] + copy2['mathematics'] + copy2['economics'])

In [271]:
copy2 = copy2.drop(columns = ['quantitative','mathematics','economics'])

In [273]:
len(list(copy2))

34

In [275]:
copy2 = copy2.drop(columns = ['location','newlocation','city','state'])

In [276]:
len(list(copy2))

30

In [277]:
list(copy2)

['Unnamed: 0',
 'position',
 'company',
 'description',
 'reviews',
 'title_category',
 'python',
 'size',
 'excel',
 'sql',
 'java',
 'scala',
 'spark',
 'machine learning',
 'data analysis',
 'data visualization',
 'modeling',
 'statistical analysis',
 'optimization',
 'artificial intelligence',
 'masters',
 'phd',
 'bachelor',
 'high school',
 'No degree requirement',
 'computer science',
 'statistics',
 'science',
 'engineer',
 'r']

In [278]:
copy2.to_csv('finalfinalclean.csv')